In [3]:
from openai import OpenAI
import spacy
spacy.prefer_gpu()
spacy_model = "en_core_web_lg"
nlp = spacy.load(spacy_model)



# use your own key
api_key = ""
client = OpenAI(api_key=api_key)
model_id = "gpt-4-1106-preview"


# $0.01 / 1K tokens, Recommend this, more precise and can input large transcripts !!!!!!!!
if model_id == "gpt-4-1106-preview":
    word_limit = 128000
    
# $0.0010 / 1K tokens
elif model_id == "gpt-3.5-turbo-1106":
    word_limit = 16385

    

file_path = r"C:\Users\ee527\Desktop\Big Data Analytics\final_project\podcast_transcript\2\The  Election Indictment Read by Ali Velshi.txt"
with open(file_path, 'r') as file:
    file_contents = file.read()
    
print("Using OpenAI model: ", model_id)
print("Using spaCy model: ", spacy_model)

Using OpenAI model:  gpt-4-1106-preview
Using spaCy model:  en_core_web_lg


### Pre-processing

In [2]:
doc = nlp(file_contents)
sentences = []
for sentence in doc.sents:
    sentences.append(sentence.text)
    
print("Total sentences in the transcript: ", len(sentences))

Total sentences in the transcript:  519


In [19]:
print(sentences)

[' Hello and welcome to a special episode of Prosecuting Donald Trump.', "I'm Ali Velshi.", 'For the third time in just four months, the former president has been criminally charged, now for the second time at the federal level.', 'But legal experts say this indictment is the most serious one yet for Trump, and the most important for our democracy.', 'The DOJ charging Trump with four counts in connection with his efforts to overturn the 2020 election results, including conspiracy to defraud the United States, conspiracy to obstruct an official proceeding, obstruction of, and attempt to obstruct an official proceeding, and conspiracy against rights.', 'The 45-page indictment also lists six unindicted co-conspirators who Trump allegedly enlisted to assist him in efforts to overturn the will of the voters.', 'The former president, of course, denies any wrongdoing.', 'As I did with the last federal indictment, I want to read you the entire document so that you have every detail of what exa

### GPT API

In [4]:
# prompt engineer here, need to change here 
number = 15

prompt = f"Step1. Read this transcript below first: \n{file_contents}\nStep2. Now give me {number} questions and the precise reference sentence you find in the transcript. Don't ask same questions.\nRule1. There are some advertisement inside this transcript, you need to recognize them and don't use those information to ask question.\nRule2. You musn't paraphrase and change words, structure in the reference sentence. The reference sentence must be completely same and can find in the transcript.\nRule3. Your response should be this format below: \nQuestion: \nReference Sentence: \n\nQuestion: \nReference Sentence: \n\n"


print(prompt)


messages = [
    
        {"role": "user", "content": prompt}

    ]

Step1. Read this transcript below first: 
 Hello and welcome to a special episode of Prosecuting Donald Trump. I'm Ali Velshi. For the third time in just four months, the former president has been criminally charged, now for the second time at the federal level. But legal experts say this indictment is the most serious one yet for Trump, and the most important for our democracy. The DOJ charging Trump with four counts in connection with his efforts to overturn the 2020 election results, including conspiracy to defraud the United States, conspiracy to obstruct an official proceeding, obstruction of, and attempt to obstruct an official proceeding, and conspiracy against rights. The 45-page indictment also lists six unindicted co-conspirators who Trump allegedly enlisted to assist him in efforts to overturn the will of the voters. The former president, of course, denies any wrongdoing. As I did with the last federal indictment, I want to read you the entire document so that you have every

In [21]:
# initial gpt and start inference
# dont need to change here
completion = client.chat.completions.create(
  model=model_id,
  messages=messages,
)

response = completion.choices[0].message.content
print(response)

Question: Who is the host of the special episode of Prosecuting Donald Trump?
Reference Sentence: Hello and welcome to a special episode of Prosecuting Donald Trump. I'm Ali Velshi.

Question: How many times has the former president been criminally charged in the past four months according to the transcript?
Reference Sentence: For the third time in just four months, the former president has been criminally charged.

Question: What are the four counts the DOJ charging Trump with?
Reference Sentence: The DOJ charging Trump with four counts in connection with his efforts to overturn the 2020 election results, including conspiracy to defraud the United States, conspiracy to obstruct an official proceeding, obstruction of, and attempt to obstruct an official proceeding, and conspiracy against rights.

Question: How many unindicted co-conspirators are listed in the indictment?
Reference Sentence: The 45-page indictment also lists six unindicted co-conspirators who Trump allegedly enlisted t

### Post-processing

In [22]:
# test question or answer

import string
def remove_punctuation(sentence):
    
    punctuation = string.punctuation
    
    while sentence and sentence[0] in punctuation:
        sentence = sentence[1:]
    
    while sentence and sentence[-1] in punctuation:
        sentence = sentence[:-1]
    
    return sentence

    
lines = [line.strip() for line in response.split("\n") if line.strip()]


questions_and_answers = []
for line in lines:
  

    if "Question:" in line:
            print("q:")
            
            colon_index = line.index(':')

            # Extract the sentence after the first colon
            question = line[colon_index + 1:].strip()
            print(question)
            
            
    elif "Reference Sentence:" in line:
        print("a:")
        
        colon_index = line.index(':')

        # Extract the sentence after the first colon
        answer = line[colon_index + 1:].strip()
        
        # clean response here
        
        # clean head and tail punctuation marks
        answer = remove_punctuation(answer)
        
        # clean "..."
        answer = answer.replace("...", "")
        
        print(answer)
     
        questions_and_answers.append((question, answer))
        
    else:
        continue


q:
Who is the host of the special episode of Prosecuting Donald Trump?
a:
Hello and welcome to a special episode of Prosecuting Donald Trump. I'm Ali Velshi
q:
How many times has the former president been criminally charged in the past four months according to the transcript?
a:
For the third time in just four months, the former president has been criminally charged
q:
What are the four counts the DOJ charging Trump with?
a:
The DOJ charging Trump with four counts in connection with his efforts to overturn the 2020 election results, including conspiracy to defraud the United States, conspiracy to obstruct an official proceeding, obstruction of, and attempt to obstruct an official proceeding, and conspiracy against rights
q:
How many unindicted co-conspirators are listed in the indictment?
a:
The 45-page indictment also lists six unindicted co-conspirators who Trump allegedly enlisted to assist him in efforts to overturn the will of the voters
q:
What did the defendant, Donald J. Trump,

In [23]:
# sentence matching

for qa in questions_and_answers:
    
        question, answer = qa
        
        print("gpt question: ", question)
        print("gpt response: ", answer)
        
        ans_start = file_contents.find(answer)
        
        # first word use lowercase
        if ans_start == -1:
          
          lowercase_answer = answer[0].lower() + answer[1:]
          ans_start = file_contents.find(lowercase_answer)
          
          
          # determine by sentence similarity (spacy)
          if ans_start == -1:
              
              similarities = []
              
              target_doc = nlp(answer)
              for sentence in sentences:
                  
                  doc = nlp(sentence)
                  similarity = doc.similarity(target_doc)
                  similarities.append(similarity)
              
              max_similarity = max(similarities)
              max_similarity_index = similarities.index(max_similarity)
              
              
              print("similarity: ", max(similarities))
              print("transcript: ", sentences[max_similarity_index])
              
              if max_similarity >= 0.95:
                  
                  ans_start = file_contents.find(sentences[max_similarity_index])
                  
              else:
                  print("Skip this error gpt response")
                  ans_start = -1
              
              
              
          else:
              print("Find! The first word in transcript is lowercase.")
              
        else:
            print("Find! Completely match.")
              
              
        print()


gpt question:  Who is the host of the special episode of Prosecuting Donald Trump?
gpt response:  Hello and welcome to a special episode of Prosecuting Donald Trump. I'm Ali Velshi
Find! Completely match.

gpt question:  How many times has the former president been criminally charged in the past four months according to the transcript?
gpt response:  For the third time in just four months, the former president has been criminally charged
Find! Completely match.

gpt question:  What are the four counts the DOJ charging Trump with?
gpt response:  The DOJ charging Trump with four counts in connection with his efforts to overturn the 2020 election results, including conspiracy to defraud the United States, conspiracy to obstruct an official proceeding, obstruction of, and attempt to obstruct an official proceeding, and conspiracy against rights
Find! Completely match.

gpt question:  How many unindicted co-conspirators are listed in the indictment?
gpt response:  The 45-page indictment als

### test spacy

In [ ]:
import spacy
spacy.prefer_gpu()

nlp = spacy.load("en_core_web_lg")

your_sentence = "Well, it's been a truly historic week as a Manhattan Grand jury voted to indict former President Donald Trump."

sentences = [
    "Natural language processing (NLP) is a field of computer science, artificial intelligence.",
    "NLP techniques are used to analyze, understand, and generate human language in a valuable way.",
    "It focuses on the interaction between computers and humans through natural language.",
    "Well, it's been a truly historic week as a Manhattan Grand jury voted to in Diteformer President Donald Trump."
  
]


target_doc = nlp(your_sentence)
for sentence in sentences:
    doc = nlp(sentence)
    similarity = doc.similarity(target_doc)
    print(similarity)

In [ ]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

path = r"C:\Users\ee527\Desktop\Big Data Analytics\final_project\podcast_transcript\4\5 million In damages.txt"


with open(path, "r", encoding="utf-8") as file:
    text = file.read()

aaa = text.split(".")
print(len(aaa))

doc = nlp(text)

sentence_list = []
for sentence in doc.sents:
   
    sentence_list.append(sentence.text)
print(len(sentence_list))


In [ ]:
print(aaa)

In [ ]:
print(sentence_list[0])

In [ ]:
print(sentence_list[0])

your_sentence = "Hello and welcome."


target_doc = nlp(your_sentence)
doc = nlp(sentence_list[0])
similarity = doc.similarity(target_doc)
print(similarity)